# Plotting the data from BIA survey

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
HIST_DIR = 'tiny_histograms/'
SUMMARY_DIR = 'summary_histograms/'

## Load the data

In [ ]:
# the whole excel sheet
df_all = pd.read_excel("BIAsurvey.xlsx")

# just the expert evaluation
df = df_all.head(7).copy()
#df['Productivity [Soil-based rooftop farming]'] = df['Productivity [Soil-based rooftop farming]'].astype('float')
df

## Plotting small histograms per criterion

In [ ]:
SPECIAL_CHARS = [" ", "[", "]", "(", ")", "-", ",", ".", ":"]
def remove_special_chars(astring):
    for spec_char in SPECIAL_CHARS:
        astring = astring.replace(spec_char, "")
    return astring

In [ ]:
BINS = [0.6, 1.4, 1.6, 2.4, 2.6, 3.4, 3.6, 4.4, 4.6, 5.4]
COLOR_RGB = [21,137,172]
SIZE = (1, 1)

def plot_histogram(column):
    fig, ax = plt.subplots(figsize=SIZE)
    ax.hist(df[column], bins=BINS, color=[x/256 for x in COLOR_RGB])
    ax.set_ylim([0,5])
    ax.set_xlim([0.6, 5.4])
    ax.set_xticks([])
    ax.set_yticks([])
    plt.axis('off')
    filename = remove_special_chars(column)+'.pdf'
    plt.savefig(HIST_DIR+filename)
    # uncomment to crop histograms
    #!pdfcrop.sh -t "0 9 0 0" $filename

In [ ]:
#alist = []
#for column in list(df):
#    alist.append(df[column].value_counts().max())
#print(max(alist))

In [ ]:
# plot tiny histograms
for column in list(df):
    try:
        plot_histogram(column)
    except:
        pass


## Plotting of summary statistics

In [ ]:
all_aspects = {
    'social': [
        'Productivity',
        'Food quality',
        'Impact on communities',
        #'Amount of human resources needed',
    ],
    'spatial' : [
        'Possibility to adapt to existing structure',
        'Impact on urban space',
        'Benefits from building synergies',
    ],
    'environmental' : [
        'Use of fertilizers',
        'Energy consumption',
        'Water consumption',
        'Biodiversity enhancement',
        'Amount of CO2 emissions',
        'Amount of possible varieties',
    ],
              }

In [ ]:
methods = ['[Soil-based rooftop farming]', '[Hydroponics]', '[Aquaponics]', '[Aeroponics]']

In [ ]:
def list_of_columns_by_aspect(aspect_list):
    alist = []
    for column in list(df):
        for criterion in aspect_list:
            if criterion in column:
                alist.append(column)
    return alist

In [ ]:
def cumulate_by_method(method, alist):
    return np.array([df[alist_element].values for alist_element in alist if method in alist_element], dtype=np.int).flatten()

In [ ]:
def average_by_method(method, alist):
        return np.mean([df_all[x].iloc[13] for x in alist if method in x], dtype=np.float)

In [ ]:
BINS = [0.6, 1.4, 1.6, 2.4, 2.6, 3.4, 3.6, 4.4, 4.6, 5.4]
bins_alternative = [0.5, 1.5, 2.5, 3.5, 4.5, 5.5]
COLOR_BLUE = [21,137,172]
COLOR_GREEN = [1, 110, 48]
COLOR_RED = [137, 2, 30]
colors = {'social': COLOR_RED,
         'spatial': COLOR_BLUE,
         'environmental' : COLOR_GREEN,
         }

SIZE_BIG = (5, 5)

def big_histogram(notes, method, aspect, mean):
    fig, ax = plt.subplots(figsize=SIZE_BIG)
    ax.hist(notes, bins=bins_alternative, color=[x/256 for x in colors[aspect]], normed=True)
    ax.set_frame_on(False)
    #ax.set_yticks([0.25, 0.50, 0.75])
    #ax.set_yticklabels([0.25, 0.50, 0.75], fontsize=16)
    _, unique_counts = np.unique(notes, return_counts=True)
    max_hist = np.max(unique_counts)/len(notes)
    ax.set_yticks([max_hist])
    ax.set_yticklabels([np.round(max_hist, decimals=2)], fontsize=20)
    ax.set_ylim([0,0.75])
    ax.set_xlim([0.6, 5.4])
    ax.set_xticklabels(['1', np.round(mean, decimals=2), '5'], fontsize=20)
    ax.set_xticks([1,mean, 5])
    #ax.tick_params(axis="y", direction="in", length=6, width=2)
    plt.axvline(x=mean, ymin=0, ymax=max_hist/0.75, c='black')
    #ax.set_title("{0}, {1} ".format(method, aspect))
    plt.savefig(SUMMARY_DIR+"{0},{1}.pdf".format(method, aspect), bbox_inches='tight')

In [ ]:
for aspect in all_aspects:
    for method in methods:
        big_histogram(cumulate_by_method( method, list_of_columns_by_aspect(all_aspects[aspect])), method, aspect,
                     mean=average_by_method(method, list_of_columns_by_aspect(all_aspects[aspect])))